<a href="https://colab.research.google.com/github/dh266/Beach_House/blob/main/genre_classification_image1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#genre_classification image

###Cài đặt gói, thư viện

In [1]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import json
import re
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss

import sklearn.preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
import glob
import csv
import cv2
import random
from PIL import Image
from itertools import product

###thêm1

In [3]:
!gdown 1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD

Downloading...
From: https://drive.google.com/uc?id=1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD
To: /content/ml1m.zip
100% 105M/105M [00:01<00:00, 75.9MB/s]


In [4]:
!unzip -qq ml1m.zip -d ml1m

In [5]:

file_id = '1oXvQ9bBc_VkzYQz7Q8Go65UZF4MvNXeM'
!gdown https://drive.google.com/uc?id={file_id}

Downloading...
From: https://drive.google.com/uc?id=1oXvQ9bBc_VkzYQz7Q8Go65UZF4MvNXeM
To: /content/ml1m_images_256x256.zip
100% 52.8M/52.8M [00:00<00:00, 102MB/s]


In [6]:
!unzip -qq ml1m_images_256x256.zip -d ml1m_images_256x256

In [7]:
# You need extract file ml1m.zip to folder ml1m before run code
import pandas
users = pandas.read_csv('ml1m/content/dataset/users.dat', sep='::',
                        engine='python',
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pandas.read_csv('ml1m/content/dataset/ratings.dat', engine='python',
                          sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies_train = pandas.read_csv('ml1m/content/dataset/movies_train.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_test = pandas.read_csv('ml1m/content/dataset/movies_test.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_train['genre'] = movies_train.genre.str.split('|')
movies_test['genre'] = movies_test.genre.str.split('|')

users.age = users.age.astype('category')
users.gender = users.gender.astype('category')
users.occupation = users.occupation.astype('category')
ratings.movieid = ratings.movieid.astype('category')
ratings.userid = ratings.userid.astype('category')


In [8]:
# movies_train = pandas.read_csv('ml1m/dataset/movies_train.dat', engine='python',
#                          sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')

folder_img_path = 'ml1m/content/dataset/ml1m-images'
movies_train['id'] = movies_train.index
movies_train.reset_index(inplace=True)
movies_train['img_path'] = movies_train.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_train

,movieid,title,genre,id,img_path
0,1650,Washington Square (1997),[Drama],1650,ml1m/content/dataset/ml1m-images/1650.jpg
1,185,"Net, The (1995)","[Sci-Fi, Thriller]",185,ml1m/content/dataset/ml1m-images/185.jpg
2,1377,Batman Returns (1992),"[Action, Adventure, Comedy, Crime]",1377,ml1m/content/dataset/ml1m-images/1377.jpg
3,3204,"Boys from Brazil, The (1978)",[Thriller],3204,ml1m/content/dataset/ml1m-images/3204.jpg
4,1901,Dear Jesse (1997),[Documentary],1901,ml1m/content/dataset/ml1m-images/1901.jpg
...,...,...,...,...,...
3101,2539,Analyze This (1999),[Comedy],2539,ml1m/content/dataset/ml1m-images/2539.jpg
3102,3038,"Face in the Crowd, A (1957)",[Drama],3038,ml1m/content/dataset/ml1m-images/3038.jpg
3103,1832,Heaven's Burning (1997),"[Action, Drama]",1832,ml1m/content/dataset/ml1m-images/1832.jpg
3104,657,Yankee Zulu (1994),"[Comedy, Drama]",657,ml1m/content/dataset/ml1m-images/657.jpg


In [9]:
folder_img_path = 'ml1m/content/dataset/ml1m-images'
movies_test['id'] = movies_test.index
movies_test.reset_index(inplace=True)
movies_test['img_path'] = movies_test.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_test

,movieid,title,genre,id,img_path
0,3397,"Great Muppet Caper, The (1981)","[Children's, Comedy]",3397,ml1m/content/dataset/ml1m-images/3397.jpg
1,2067,Doctor Zhivago (1965),"[Drama, Romance, War]",2067,ml1m/content/dataset/ml1m-images/2067.jpg
2,2651,Frankenstein Meets the Wolf Man (1943),[Horror],2651,ml1m/content/dataset/ml1m-images/2651.jpg
3,2989,For Your Eyes Only (1981),[Action],2989,ml1m/content/dataset/ml1m-images/2989.jpg
4,3415,"Mirror, The (Zerkalo) (1975)",[Drama],3415,ml1m/content/dataset/ml1m-images/3415.jpg
...,...,...,...,...,...
772,2309,"Inheritors, The (Die Siebtelbauern) (1998)",[Drama],2309,ml1m/content/dataset/ml1m-images/2309.jpg
773,2421,"Karate Kid, Part II, The (1986)","[Action, Adventure, Drama]",2421,ml1m/content/dataset/ml1m-images/2421.jpg
774,3255,"League of Their Own, A (1992)","[Comedy, Drama]",3255,ml1m/content/dataset/ml1m-images/3255.jpg
775,974,Algiers (1938),"[Drama, Romance]",974,ml1m/content/dataset/ml1m-images/974.jpg


### Creating multi-hot encoded genre vectors

sử dụng vector nhị phân mã hóa  hiện diện của nhiều thể loại
Ví dụ, giả sử bạn có ba thể loại: Hành động, Kịch tính, và Hài
phim thuộc cả hai thể loại Hành động và Hài, vector được mã hóa multi-hot có thể trông như sau:
Hành động: 1
Kịch tính: 0
Hài: 1

####add 2

In [10]:
mlb = MultiLabelBinarizer()#train
multihot_temp = mlb.fit_transform(movies_train["genre"])

In [11]:
mlb2 = MultiLabelBinarizer()
multihot_temp2 = mlb.fit_transform(movies_test["genre"])

In [12]:
#    'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime',
#   'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
#   'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
#train

multihot = [0] * len(multihot_temp)

item = 0
for vec in multihot_temp:

    new_vec = np.empty([18])
    for i in range(18):
        new_vec[i] = 0


    index = 0
    for i in range(18):
        if vec[i] == 1:
            # Handle special cases if needed
            new_vec[index] = 1
            index += 1

        if vec[i] == 0:
            index += 1

    multihot[item] = new_vec
    item += 1

In [13]:
#    'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime',
#   'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
#   'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
#test

multihot2 = [0] * len(multihot_temp2)

item2 = 0
for vec in multihot_temp2:

    new_vec = np.empty([18])
    for i in range(18):
        new_vec[i] = 0


    index = 0
    for i in range(18):
        if vec[i] == 1:
            # Handle special cases if needed
            new_vec[index] = 1
            index += 1

        if vec[i] == 0:
            index += 1

    multihot2[item2] = new_vec
    item2 += 1

####//

In [14]:
genres_df = pd.DataFrame({"multihot":[multihot_temp.astype(int)]}, index = movies_train.index)
movies_train = pd.concat([movies_train, genres_df], axis=1 )
print(mlb.classes_)

['Action' 'Adventure' 'Animation' "Children's" 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'Musical' 'Mystery'
 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western']


In [15]:
genres_df2 = pd.DataFrame({"multihot":[multihot_temp2.astype(int)]}, index = movies_test.index)
movies_test = pd.concat([movies_test, genres_df2], axis=1 )
print(mlb.classes_)

['Action' 'Adventure' 'Animation' "Children's" 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'Musical' 'Mystery'
 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western']


####//

In [16]:
#create a dictionary with multi-hot encoded vectors;
#multihot_dict = {movies_train.index.tolist()[i] : multihot[i] for i in range(0, len(multihot))}
#train
multihot_dict = {str(index): multihot[index] for index in movies_train.index.tolist()}

In [17]:
#test
multihot_dict2 = {str(index): multihot2[index] for index in movies_test.index.tolist()}

## Hyper Parameters

In [18]:
#training controls
batch_size = 1
number_of_labels = 18
epochs = 10
training_size = 0.7
learning_rate = 0.03 #0.5 #0.01 #0.001
dropout = [0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.15]
# input image dimensions
img_rows, img_cols = 150, 150

### Passing the images through a convolutional network

In [19]:

# the data holders
x_test = []
x_train = []
y_test = []
y_train = []

#images need to have the same size!!
flist=glob.glob('ml1m_images_256x256/ml1m_images_256x256/*.jpg')
random.shuffle(flist)

#length=int(len(flist)*training_size)
i = 0

genre_count = np.empty([number_of_labels])
for l in range(number_of_labels):
    genre_count[l] = 0

#create lists with input data (images) and output data (multi-hot encoded genre vectors)
for filename in flist:

    # Trích xuất IMDb ID từ tên tệp
    movieid = os.path.splitext(os.path.basename(filename))[0]  # Lấy phần tên tệp không có phần mở rộng (.jpg)



    if movieid in multihot_dict:

        img = np.array(cv2.imread(filename))
        img = np.swapaxes(img, 2, 0)
        img = np.swapaxes(img, 2, 1)

        genre_arr = np.array(multihot_dict[movieid])

        #if i < length:
        x_train.append(img)
        y_train.append(genre_arr)
        #else:
        #   x_test.append(img)
        #   y_test.append(genre_arr)

        i += 1

        # Update genre_count for training set
        genre_count += genre_arr

    if movieid in multihot_dict2:

        img = np.array(cv2.imread(filename))
        img = np.swapaxes(img, 2, 0)
        img = np.swapaxes(img, 2, 1)

        genre_arr2 = np.array(multihot_dict2[movieid])

        #if i < length:
        #    x_train.append(img)
        #    y_train.append(genre_arr)
        #else:
        x_test.append(img)
        y_test.append(genre_arr)

        i += 1

        # Update genre_count for training set
        genre_count += genre_arr2

classes = ["Action", "Adventure", "Animation", "Children\'s", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
img_sum = 0
for c in range(17):
    print(classes[c] + ": "+ str(genre_count[c]))
    img_sum += genre_count[c]


Action: 405.0
Adventure: 237.0
Animation: 88.0
Children's: 209.0
Comedy: 992.0
Crime: 175.0
Documentary: 104.0
Drama: 1314.0
Fantasy: 56.0
Film-Noir: 37.0
Horror: 281.0
Musical: 91.0
Mystery: 85.0
Romance: 379.0
Sci-Fi: 233.0
Thriller: 408.0
War: 111.0


#//

####//

In [20]:

#converting the data from lists to numpy arrays
x_train=np.asarray(x_train,dtype=float)
x_test=np.asarray(x_test,dtype=float)
y_train=np.asarray(y_train,dtype=float)
y_test=np.asarray(y_test,dtype=float)

#scaling down the RGB data
x_train /= 255
x_test /= 255

#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)


x_train shape: (2557, 3, 256, 256)
2557 train samples
640 test samples


### DenseNet-121

In [21]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Linear(1024, number_of_labels),
    # nn.Linear(1024, 512),
    # nn.Dropout(p=0.1),
    # nn.ReLU(),
    # nn.Linear(512, 28),
    nn.Sigmoid()
)


model.double()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"model loaded on {device}")
model.to(device) #hopefully runs model on cuda core.
print(model)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 133MB/s]


model loaded on cuda:0
DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_st

## Loss function

In [22]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0) #TODO: check best params here

## Train and Test functions

In [23]:
# Function to save the model
def saveModel():
    path = "./trained_model.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    model.eval() #TODO: check model.train() which is used along model.eval()
    accuracy = 0.0
    total = 0.0

    classes = ["Action", "Adventure", "Animation", "Children\'s", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

    class_correct = list(0. for i in range(number_of_labels))
    class_total = list(0. for i in range(number_of_labels))

    with torch.no_grad():
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(f"model testing on {device}")
        some_accuracy_measure = 0
        final_accuracy = 0
        for i, (images, labels) in enumerate(test_loader,0):
            sum = 0
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            outputs = model(images)
            for j, label in enumerate(labels):
              n = sum + label.sum().item()
              # get the top n values of outputs
              _, predicted = torch.topk(outputs[j], int(n)) #TODO: find best threshold and calc F1.

              correct_predictions = 0
              for _, k in enumerate(predicted):
                class_total[k] += 1
                if(label[k].item() == 1):
                    correct_predictions += 1
                    class_correct[k] +=1
                some_accuracy_measure = correct_predictions/n
            final_accuracy += some_accuracy_measure
            # the label with the highest energy will be our prediction
            total += labels.size(0)
    print("Total: ", total)
    # compute the accuracy over all test images
    accuracy = final_accuracy * 100 /total
    print("Accuracy;: ", accuracy)

    for i in range(number_of_labels):
        if class_total[i]!=0:
            print('Accuracy of %5s : %2d %%' % (
                    classes[i], 100 * class_correct[i] / class_total[i]))

    model.train()
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):

    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        print("Starting epoch", epoch)
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):

            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            #if i % 1000 == 0:
                # print every 1000 (twice per epoch)
        print('[%d] loss: %.3f' %
            (epoch , running_loss / 1000))
                # zero the loss
        running_loss = 0.0

        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy() #BUG: why is this in for loop?
        print('For epoch', epoch,'accuracy over test set %d %%' % (accuracy))

        #  save the model if the accuracy is the best
        #if accuracy > best_accuracy:
         #   saveModel()
          #  best_accuracy = accuracy

## Functions to display sample output

In [26]:
# Function to show the images
def imageshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# Function to test the model with a batch of images and show the labels predictions
def testBatch():
    # get batch of images from the test DataLoader
    images, labels = next(iter(test_loader))

    # show all images as one image grid
    imageshow(torchvision.utils.make_grid(images))

    # Show the real labels on the screen
    print('Real labels: ', ' '.join('%5s' % classes[labels[j]]
                               for j in range(batch_size)))

    # Let's see what if the model identifiers the  labels of those example
    outputs = model(images)

    # We got the probability for every 10 labels. The highest (max) probability should be correct label
    _, predicted = torch.max(outputs, 1)

    # show the predicted labels
    print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(batch_size)))

## Run Model

In [27]:
train(3)
print("Training done.")

The model will be running on cuda:0 device
Starting epoch 0
[0] loss: 1.872
model testing on cuda:0
Total:  640.0
Accuracy;:  37.58072916666668
Accuracy of Action : 48 %
Accuracy of Adventure : 50 %
Accuracy of Animation : 13 %
Accuracy of Comedy : 33 %
Accuracy of Crime :  7 %
Accuracy of Documentary :  0 %
Accuracy of Drama : 40 %
Accuracy of Fantasy : 10 %
Accuracy of Film-Noir :  0 %
Accuracy of Musical : 33 %
Accuracy of Mystery :  0 %
Accuracy of Romance : 22 %
Accuracy of Sci-Fi : 100 %
For epoch 0 accuracy over test set 37 %
Starting epoch 1
[1] loss: 1.872
model testing on cuda:0
Total:  640.0
Accuracy;:  38.09635416666667
Accuracy of Action : 56 %
Accuracy of Adventure : 40 %
Accuracy of Animation : 30 %
Accuracy of Comedy : 41 %
Accuracy of Crime :  0 %
Accuracy of Documentary :  0 %
Accuracy of Drama : 40 %
Accuracy of Fantasy : 22 %
Accuracy of Film-Noir :  0 %
Accuracy of Musical :  0 %
Accuracy of Mystery :  0 %
Accuracy of Romance : 22 %
Accuracy of Sci-Fi :  0 %
For ep

In [28]:
model.eval()
accuracy = 0.0
total = 0.0

classes = ["Action", "Adventure", "Animation", "Children\'s", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

class_correct = list(0. for i in range(number_of_labels))
class_total = list(0. for i in range(number_of_labels))

with torch.no_grad():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"model testing on {device}")
    some_accuracy_measure = 0
    final_accuracy = 0
    for i, (images, labels) in enumerate(test_loader,0):
        sum = 0
        images = Variable(images.to(device))
        labels = Variable(labels.to(device))
        outputs = model(images)
        for j, label in enumerate(labels):
            n = sum + label.sum().item()
            # get the top n values of outputs
            _, predicted = torch.topk(outputs[j], int(n)) #TODO: find best threshold and calc F1.
            _, groundtruth = torch.topk(labels[j], int(n))

            correct_predictions = 0
            for _, k in enumerate(predicted):

                if(label[k].item() == 1):
                    correct_predictions += 1
                some_accuracy_measure = correct_predictions/n
                if (0):
                    imageshow(images[j])
                    print(outputs[j])
                    print(labels[j])
                    print(some_accuracy_measure)
            for l in range(number_of_labels):

                if (label[l].item() == 1):
                    class_total[l] += 1
                    if (l in predicted):
                        class_correct[l] +=1
        final_accuracy += some_accuracy_measure

        total += labels.size(0)

print("Total: ", total)
# compute the accuracy over all test images
accuracy = final_accuracy * 100 /total
print("Accuracy;: ", accuracy)

model testing on cuda:0
Total:  640.0
Accuracy;:  37.57552083333334
